### Load Packages

In [2]:
import os
import openai
from scipy.spatial import distance
import plotly.express as px
from sklearn.cluster import KMeans
from umap import UMAP

In [41]:
from dotenv import load_dotenv

load_dotenv('./dev.env')

print(os.getenv("OPENAI_API_KEY"))


sk-0Zc7LOdabqS5DtOIkENUT3BlbkFJ941DWQYDHCv6mK9I


In [47]:
from dotenv import load_dotenv

load_dotenv(dotenv_path='dev.env')

print(os.getenv("OPENAI_API_KEY"))


sk-0Zc7LOdabqS5DtOIkENUT3BlbkFJ941DWQYDHCv6mK9I


In [51]:
import openai
client = openai.OpenAI(
   api_key=openai.api_key
)

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input=[text], model=model).data[0].embedding

### Load Dataset

In [25]:
import pandas as pd

review_df = pd.read_csv("data4.csv", encoding='latin1')
df = review_df[["message"]]
df.count()

message    78
dtype: int64

### Get Text Embeddings from Error logs

In [69]:
# Assuming get_embedding function is defined elsewhere and correctly takes a text input and a model name
df['embedding'] = df['message'].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

# Make the index start from 0 and ensure the change is saved
df = df.reset_index(drop=True)

df.head(10)

,message,embedding
0,"""Faulting messaging object due to an error. Na...","[-0.015120312571525574, 0.003530361922457814, ..."
1,"""Aborting messaging object. Name = srikandi:qu...","[-0.02315927855670452, 0.04029439762234688, 0...."
2,"""Aborting messaging object. Name = srikandi:Qu...","[-0.024547232314944267, 0.05129839479923248, 0..."
3,"""Faulting messaging object due to an error. Na...","[-0.016832994297146797, 0.0037243624683469534,..."
4,"""Faulting messaging object due to an error. Na...","[-0.026517968624830246, 0.0393458716571331, 0...."
5,"""An exception was handled. TrackingId: 9531470...","[-0.01762230694293976, 0.03339536115527153, 0...."
6,"""Aborting messaging object. Name = srikandi:qu...","[-0.025672094896435738, 0.04192393273115158, 0..."
7,"""Aborting messaging object. Name = srikandi:Qu...","[-0.026734808459877968, 0.051062215119600296, ..."
8,"""Aborting messaging object. Name = bl-uat-mode...","[-0.0180745217949152, 0.06595976650714874, 0.0..."
9,"""Aborting messaging object. Name = bl-uat-mode...","[-0.011245463974773884, 0.07717191427946091, -..."


In [77]:
df.to_csv('output/embeddings.csv', index=False)


### Visualize Clusters

In [70]:
kmeans = KMeans(n_clusters=8)


In [71]:
import numpy as np
embeddings = np.stack(df['embedding'].values)


In [73]:
kmeans.fit(embeddings)

KMeans()

In [75]:
#Reduce Dimensionality
reducer = UMAP()
embeddings_2d = reducer.fit_transform(embeddings.tolist())

In [76]:
#Visualize Cluster
fig = px.scatter(x=embeddings_2d[:, 0], y=embeddings_2d[:, 1], color=kmeans.labels_)
fig.show()

## Fine-Tuning + Prompt Engineering

### Assigned Label Clusters

In [78]:
import numpy as np

matrix = np.stack(df["embedding"].values)

from sklearn.cluster import KMeans

n_clusters = 8

kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=42)
kmeans.fit(matrix)

labels = kmeans.labels_
df["Cluster"] = labels.astype(str)  # Convert labels to string type

In [ ]:
import seaborn as sns
import matplotlib

In [79]:
df['Cluster'] = df['Cluster'].astype(int)

In [81]:
import openai

# Number of error logs to review per cluster
logs_per_cluster = 10

for i in range(n_clusters):
    print(f"Cluster {i} Anomalies:", end=" ")

    # Filter the DataFrame for the current cluster
    cluster_df = review_df[review_df.Cluster == i]

    # Check if there are enough logs in the cluster to sample from
    if len(cluster_df) >= logs_per_cluster:
        # Extract a sample of error logs from each cluster
        error_logs = "\n".join(
            cluster_df["message"]
            .sample(n=logs_per_cluster, random_state=42)
            .values
        )
    else:
        # If not enough logs, use whatever is available
        error_logs = "\n".join(cluster_df["message"].values)
    
    # Create a prompt for GPT-4-5 turbo to analyze the error logs
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        #Fine-Tune the Message Prompt
        messages=[
            {"role": "system", "content": "You are providing data analysis on Microsoft Azure Service Bus error log messages. This cloud service is designed to facilitate communication between applications and services. In analyzing Azure Service Bus error logs, common issues fall into categories such as permissions, connectivity, sender, receiver, processor, and transaction-related problems. These encompass handling Service Bus exceptions, managing access rights, addressing network connection challenges, troubleshooting message sending and receiving difficulties, resolving processor operation concerns, and navigating transaction complexities. Your primary role is identifying log error patterns clustered around a suspected anomaly and summarizing what the clusters of log messages represent."},
            {"role": "user", "content": f'Error Logs:\n"""\n{error_logs}\n"""\n\nIdentified Patterns or Anomalies:'}
        ],
        temperature=0.1,
        max_tokens=300,  # Adjusted max_tokens for potentially more detailed analysis
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    print(response.choices[0].message.content.strip())

    # Optionally, display additional information from the sampled logs
    # Check again if there are enough logs
    if len(cluster_df) >= logs_per_cluster:
        sample_cluster_logs = cluster_df.sample(n=logs_per_cluster, random_state=42)
    else:
        sample_cluster_logs = cluster_df

    for j in range(len(sample_cluster_logs)):
        print(sample_cluster_logs.iloc[j].get('message', '')[:70])  # Display the first part of the error log

Cluster 0 Anomalies: Based on the provided Azure Service Bus error logs, several patterns and anomalies can be identified. These patterns suggest issues related to timeouts, aborted operations, and missing messaging entities. Here's a summary of the identified patterns:

1. **Timeout Issues:**
   - Several log messages indicate that operations are being aborted due to timeouts. This suggests that the Service Bus is not able to complete certain operations within the allotted time frame. This could be due to network latency, resource constraints, or inefficient processing.
   - Example log messages:
     - "The operation did not complete within the allotted timeout of 00:01:05."
     - "The operation did not complete within the allotted timeout of 00:02:00."

2. **Aborted Operations:**
   - Multiple log entries mention that messaging objects are being aborted. This could be due to various reasons such as internal errors, resource limitations, or unexpected conditions that cause the Servi